In [ ]:
import os
import os.path as osp
import time
import datetime
import random
from PIL import Image
import numpy as np
from sklearn.metrics import accuracy_score
import torch.nn as nn
from torch.autograd import Function
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf
import torch
import pandas as pd
import torch.utils.data as data
from torch.utils.data import Dataset, TensorDataset, DataLoader, RandomSampler, SequentialSampler, WeightedRandomSampler
import logging
from torchvision import datasets,transforms, models
import tqdm
import glob
import matplotlib.image as image
from PIL import Image

In [ ]:
# Download YOLOv7 code
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7

In [ ]:
from models.yolo import Model
from utils.general import check_requirements, set_logging
from utils.google_utils import attempt_download
from utils.torch_utils import select_device
from  hubconf import custom
model = custom(path_or_model='best.pt', autoshape = True) 

model.conf = 0.1

In [ ]:
from PIL import Image

img = Image.open('/kaggle/input/hsgshackathon2021/Test_data/Test/video_60a/frame_003852.PNG')

result = model(img)
print(result.pandas().xyxy[0])
# Run evaluation
!python detect.py --weights /kaggle/input/dambong/best.pt --conf 0.1 --source /kaggle/input/hsgshackathon2021/Test_data/Test/video_60a/frame_003852.PNG

In [ ]:
import glob
from IPython.display import Image, display

i = 0
limit = 10000 # max images to print
for imageName in glob.glob('/kaggle/working/yolov7/runs/detect/exp4/frame_003852.PNG'): #assuming JPG
    if i < limit:
      display(Image(filename=imageName))
      print("\n")
    i = i + 1

In [ ]:
#TEST_DATA
from PIL import Image
test_data = np.array([])
ans = []
test_pred = []
for r, dirnames, fn in os.walk('/kaggle/input/hsgshackathon2021/Test_data/Test'):
    for dirname in sorted(dirnames):
       # print(dirname)
        cur_path = '/kaggle/input/hsgshackathon2021/Test_data/Test' + '/' + dirname
        for filename in sorted(os.listdir(cur_path)):
             ans.append((dirname+'_'+filename))
             ans[(len(ans)-1)]=ans[(len(ans)-1)].strip('.PNG')
             img = Image.open((cur_path+'/'+filename))
             if dirname =='video_20b':
                img = img.rotate(90)
             result = model(img)
             table = result.pandas().xyxy[0]
             ball = []
             hand = []
             for i in range(0, len(table)):
                    if table.name[i] == '0':
                        ball1 = []
                        ball1.append(table.xmin[i])
                        ball1.append(table.xmax[i])
                        ball1.append(table.ymin[i])
                        ball1.append(table.ymax[i])
                        ball.append(ball1)
                    elif table.name[i] == '1':
                        hand1 = []
                        hand1.append(table.xmin[i])
                        hand1.append(table.xmax[i])
                        hand1.append(table.ymin[i])
                        hand1.append(table.ymax[i])
                        hand.append(hand1)
             mx = 0
            
             for i in ball:
                    for j in (hand):
                        ballxmin = i[0]
                        ballxmax = i[1]
                        ballymin = i[2]
                        ballymax = i[3]
                        handxmin = j[0]
                        handxmax = j[1]
                        handymin = j[2]
                        handymax = j[3]
                        if ballxmin <= handxmax and handxmax <= ballxmax and (not(ballymin>handymax or handymin>ballymax)):
                            mx = 1
                        if handxmin <= ballxmax and ballxmax <= handxmax and (not(ballymin>handymax or handymin>ballymax)): 
                            mx = 1
                        if (abs(ballxmin-handxmax) <= 4) and (not(ballymin>handymax or handymin>ballymax)):
                            mx = 1
                        if abs(ballxmax-handxmin) <= 4 and (not(ballymin>handymax or handymin>ballymax)):
                            mx = 1
             test_pred.append(mx)
        
           #  img = img.resize(224, 224)

In [ ]:
submission = pd.DataFrame(data=zip(ans,test_pred),columns=["Frame","Label"])
submission.head()
print(submission)